In [37]:
from sklearn import svm
import numpy as np
import pandas as pd
import re
import random
import pickle

In [2]:
files = ['DS_1.csv', 'DS_2.csv', 'DS_3.csv', 'DS_4.csv', 'DS_5.csv', 'DS_6.csv', 'DS_7.csv', 'DS_8.csv', 
         'DS_9.csv', 'DS_10.csv', 'DS_11.csv', 'DS_12.csv', 'DS_13.csv', 'DS_14.csv', 'DS_15.csv', 'DS_16.csv',
         'DS_17.csv', 'DS_18.csv', 'DS_19.csv', 'DS_20.csv', 'DS_21.csv', 'DS_22.csv']

df = pd.DataFrame()

for file in files:
    data = pd.read_csv(file)
    df = pd.concat([df, data], axis=0)

In [3]:
df = df.drop(['Unnamed: 0'], axis = 1)
df = df.sample(frac = 1)
df

,Word,Word POS,Sense,Feature Vector,Label
35,फीका,a,pale.v.01,[0. 0. 0. 0. 0. 0. 0.],0
398,अतिरिक्त,a,extra.r.01,[0. 0. 0. 0. 0.7 0. 0. ],0
169,झुकाव,n,cant.v.01,[0. 0. 0. 0. 0. 0. 0.],0
1759,ठहराना,v,catch.n.06,[0. 0. 0. 0. 0. 0. 0.],0
783,चाक,n,wheel.n.03,[ 1. 0. 5.15555556 1. ...,0
...,...,...,...,...,...
349,स्वभाव,n,character.n.08,[1. 0. 3.01785714 1. 0...,0
143,निर्णय,n,judgment.n.02,[1. 0. 4.46666667 1. 1...,0
164,समाप्त,a,finished.a.01,[1. 0. 0.28571429 0. 0...,1
446,बचाव,n,guard.n.06,[ 1. 0. 6.66666667 1. ...,0


In [4]:
feature_vectors = []
label = []
for i in range(len(df)):
    if df['Label'].iloc[i] == 1:
        vector = df['Feature Vector'].iloc[i]
        pos = re.findall("\d*\.\d*", vector)
        vals = []
        for val in pos:
            vals.append(float(val))
        
        feature_vectors.append(np.array(vals))
        label.append(1)

In [5]:
print(len(feature_vectors))
print(len(label))

1027
1027


In [6]:
false_count = len(label)
count = 0
for i in range(len(df)):
    if count <= false_count:
        if df['Label'].iloc[i] == 0:
            vector = df['Feature Vector'].iloc[i]
            pos = re.findall("\d*\.\d*", vector)
            vals = []
            for val in pos:
                vals.append(float(val))

            feature_vectors.append(np.array(vals))
            label.append(0)
            count += 1

In [7]:
print(len(feature_vectors))
print(len(label))

2055
2055


In [8]:
random.shuffle(feature_vectors)
random.shuffle(label)
X = feature_vectors
Y = label

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

In [10]:
clf = svm.SVC()
clf.fit(X_train, Y_train)

SVC()

In [11]:
correct = 0
correct_1 = 0
for i in range(len(X_test)):
    test = [X_test[i]]
    pred = clf.predict(test)
    if pred[0] == Y_test[i]:
        correct += 1
        if pred[0] == 1:
            correct_1 += 1
        
print(correct)
print(correct_1)
print(len(X_test))
print(correct/len(X_test))

108
26
206
0.5242718446601942


In [12]:
from sklearn.metrics import f1_score
Y_pred = clf.predict(X_test)
f1_score(Y_test, Y_pred, average='binary')

0.3466666666666667

In [13]:
from sklearn.metrics import precision_score
Y_pred = clf.predict(X_test)
precision_score(Y_test, Y_pred, average='binary')

0.5531914893617021

In [14]:
from sklearn.metrics import recall_score
Y_pred = clf.predict(X_test)
recall_score(Y_test, Y_pred, average='binary')

0.2524271844660194

In [ ]:
ilename = 'svm_wordnet.sav'
pickle.dump(model, open(filename, 'wb'))

## Neural Net

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow import keras

In [19]:
np.array(X_train).shape

(1849, 7)

In [20]:
np.array(Y_train).shape

(1849,)

In [21]:
input_dim = tuple((7, ))
input_dim

(7,)

In [22]:
model = Sequential()
model.add(Dense(32, activation = 'relu', input_shape = input_dim))
model.add(Dense(16, activation='relu', kernel_regularizer = keras.regularizers.L1L2(l1=1e-5, l2=1e-4)))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                256       
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [24]:
history = model.fit(np.array(X_train), np.array(Y_train), validation_split = 0.2, epochs=300, batch_size=32)

2023-05-12 11:59:07.012083: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/300
47/47 [==============================] - 0s 2ms/step - loss: 0.7032 - accuracy: 0.4956 - val_loss: 0.7097 - val_accuracy: 0.4676
Epoch 2/300
47/47 [==============================] - 0s 604us/step - loss: 0.7002 - accuracy: 0.5037 - val_loss: 0.7097 - val_accuracy: 0.4676
Epoch 3/300
47/47 [==============================] - 0s 504us/step - loss: 0.6991 - accuracy: 0.4990 - val_loss: 0.7050 - val_accuracy: 0.4730
Epoch 4/300
47/47 [==============================] - 0s 501us/step - loss: 0.6993 - accuracy: 0.4895 - val_loss: 0.7050 - val_accuracy: 0.4649
Epoch 5/300
47/47 [==============================] - 0s 527us/step - loss: 0.6989 - accuracy: 0.5003 - val_loss: 0.7066 - val_accuracy: 0.4973
Epoch 6/300
47/47 [==============================] - 0s 509us/step - loss: 0.6977 - accuracy: 0.5024 - val_loss: 0.7091 - val_accuracy: 0.4838
Epoch 7/300
47/47 [==============================] - 0s 535us/step - loss: 0.6977 - accuracy: 0.5024 - val_loss: 0.7113 - val_accuracy: 0.4730
E

Epoch 58/300
47/47 [==============================] - 0s 481us/step - loss: 0.6941 - accuracy: 0.5436 - val_loss: 0.7061 - val_accuracy: 0.4649
Epoch 59/300
47/47 [==============================] - 0s 493us/step - loss: 0.6940 - accuracy: 0.5348 - val_loss: 0.7061 - val_accuracy: 0.4541
Epoch 60/300
47/47 [==============================] - 0s 482us/step - loss: 0.6934 - accuracy: 0.5335 - val_loss: 0.7052 - val_accuracy: 0.4622
Epoch 61/300
47/47 [==============================] - 0s 466us/step - loss: 0.6930 - accuracy: 0.5335 - val_loss: 0.7057 - val_accuracy: 0.4622
Epoch 62/300
47/47 [==============================] - 0s 496us/step - loss: 0.6942 - accuracy: 0.5287 - val_loss: 0.7060 - val_accuracy: 0.4730
Epoch 63/300
47/47 [==============================] - 0s 474us/step - loss: 0.6935 - accuracy: 0.5308 - val_loss: 0.7069 - val_accuracy: 0.4757
Epoch 64/300
47/47 [==============================] - 0s 492us/step - loss: 0.6936 - accuracy: 0.5308 - val_loss: 0.7132 - val_accuracy:

Epoch 115/300
47/47 [==============================] - 0s 506us/step - loss: 0.6923 - accuracy: 0.5301 - val_loss: 0.7076 - val_accuracy: 0.4730
Epoch 116/300
47/47 [==============================] - 0s 482us/step - loss: 0.6925 - accuracy: 0.5348 - val_loss: 0.7069 - val_accuracy: 0.4811
Epoch 117/300
47/47 [==============================] - 0s 487us/step - loss: 0.6929 - accuracy: 0.5368 - val_loss: 0.7074 - val_accuracy: 0.4757
Epoch 118/300
47/47 [==============================] - 0s 477us/step - loss: 0.6923 - accuracy: 0.5328 - val_loss: 0.7059 - val_accuracy: 0.4784
Epoch 119/300
47/47 [==============================] - 0s 474us/step - loss: 0.6913 - accuracy: 0.5423 - val_loss: 0.7057 - val_accuracy: 0.4784
Epoch 120/300
47/47 [==============================] - 0s 494us/step - loss: 0.6920 - accuracy: 0.5389 - val_loss: 0.7059 - val_accuracy: 0.4784
Epoch 121/300
47/47 [==============================] - 0s 483us/step - loss: 0.6923 - accuracy: 0.5335 - val_loss: 0.7059 - val_ac

47/47 [==============================] - 0s 518us/step - loss: 0.6907 - accuracy: 0.5409 - val_loss: 0.7080 - val_accuracy: 0.4676
Epoch 172/300
47/47 [==============================] - 0s 477us/step - loss: 0.6915 - accuracy: 0.5443 - val_loss: 0.7073 - val_accuracy: 0.4703
Epoch 173/300
47/47 [==============================] - 0s 456us/step - loss: 0.6909 - accuracy: 0.5436 - val_loss: 0.7065 - val_accuracy: 0.4811
Epoch 174/300
47/47 [==============================] - 0s 478us/step - loss: 0.6909 - accuracy: 0.5409 - val_loss: 0.7104 - val_accuracy: 0.4486
Epoch 175/300
47/47 [==============================] - 0s 454us/step - loss: 0.6914 - accuracy: 0.5389 - val_loss: 0.7106 - val_accuracy: 0.4541
Epoch 176/300
47/47 [==============================] - 0s 468us/step - loss: 0.6913 - accuracy: 0.5328 - val_loss: 0.7065 - val_accuracy: 0.4784
Epoch 177/300
47/47 [==============================] - 0s 465us/step - loss: 0.6912 - accuracy: 0.5416 - val_loss: 0.7083 - val_accuracy: 0.4676

47/47 [==============================] - 0s 475us/step - loss: 0.6902 - accuracy: 0.5450 - val_loss: 0.7104 - val_accuracy: 0.4514
Epoch 228/300
47/47 [==============================] - 0s 498us/step - loss: 0.6912 - accuracy: 0.5274 - val_loss: 0.7079 - val_accuracy: 0.4703
Epoch 229/300
47/47 [==============================] - 0s 487us/step - loss: 0.6892 - accuracy: 0.5443 - val_loss: 0.7166 - val_accuracy: 0.4703
Epoch 230/300
47/47 [==============================] - 0s 495us/step - loss: 0.6902 - accuracy: 0.5375 - val_loss: 0.7073 - val_accuracy: 0.4730
Epoch 231/300
47/47 [==============================] - 0s 479us/step - loss: 0.6908 - accuracy: 0.5443 - val_loss: 0.7080 - val_accuracy: 0.4757
Epoch 232/300
47/47 [==============================] - 0s 483us/step - loss: 0.6909 - accuracy: 0.5375 - val_loss: 0.7065 - val_accuracy: 0.4703
Epoch 233/300
47/47 [==============================] - 0s 479us/step - loss: 0.6902 - accuracy: 0.5341 - val_loss: 0.7065 - val_accuracy: 0.4811

47/47 [==============================] - 0s 507us/step - loss: 0.6900 - accuracy: 0.5375 - val_loss: 0.7085 - val_accuracy: 0.4622
Epoch 284/300
47/47 [==============================] - 0s 480us/step - loss: 0.6899 - accuracy: 0.5335 - val_loss: 0.7077 - val_accuracy: 0.4730
Epoch 285/300
47/47 [==============================] - 0s 486us/step - loss: 0.6902 - accuracy: 0.5429 - val_loss: 0.7075 - val_accuracy: 0.4757
Epoch 286/300
47/47 [==============================] - 0s 475us/step - loss: 0.6905 - accuracy: 0.5429 - val_loss: 0.7075 - val_accuracy: 0.4676
Epoch 287/300
47/47 [==============================] - 0s 476us/step - loss: 0.6899 - accuracy: 0.5510 - val_loss: 0.7101 - val_accuracy: 0.4703
Epoch 288/300
47/47 [==============================] - 0s 484us/step - loss: 0.6900 - accuracy: 0.5409 - val_loss: 0.7078 - val_accuracy: 0.4703
Epoch 289/300
47/47 [==============================] - 0s 484us/step - loss: 0.6903 - accuracy: 0.5402 - val_loss: 0.7071 - val_accuracy: 0.4757

In [25]:
prediction = model.predict(np.array(X_test))

In [26]:
prediction_discrete = (prediction > 0.5).astype(np.float32)

In [27]:
from sklearn.metrics import f1_score
f1_score(Y_test, prediction_discrete, average='binary')

0.5596330275229358

In [28]:
correct = 0
for i in range(len(prediction_discrete)):
    if prediction_discrete[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

110
206
0.5339805825242718


In [29]:
model_json = model.to_json()
with open("neural_net_wordnet.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("neural_net_wordnet.h5")

# SVR

In [30]:
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [31]:
regr = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2))
regr.fit(X_train, Y_train)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('svr', SVR(epsilon=0.2))])

In [32]:
pred = regr.predict(X_test)
prediction_discrete = (pred > 0.5).astype(np.float32)

correct = 0
for i in range(len(prediction_discrete)):
    if prediction_discrete[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

109
206
0.529126213592233


In [33]:
Y_pred = clf.predict(X_test)

In [34]:
from sklearn.metrics import f1_score
f1_score(Y_test, Y_pred, average='binary')

0.3466666666666667

## GaussianNB

In [35]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
y_pred = gnb.fit(X_train, Y_train).predict(X_test)

In [36]:
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

109
206
0.529126213592233


In [38]:
pickle.dump(gnb, open('gassiannb_wordnet.sav', 'wb'))

## MultinomialNB

In [39]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(force_alpha=True)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

111
206
0.5388349514563107


In [40]:
pickle.dump(clf, open('multinomialnb_wordnet.sav', 'wb'))

## CategoricalNB

In [41]:
from sklearn.naive_bayes import CategoricalNB
clf = CategoricalNB(force_alpha=True)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)

correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

107
206
0.5194174757281553


In [42]:
pickle.dump(clf, open('categoricalnb_wordnet.sav', 'wb'))

## DecisionTree

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train, Y_train, check_input=True)

y_pred = clf.predict(X_test)
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

108
206
0.5242718446601942


In [45]:
pickle.dump(clf, open('decisiontree_wordnet.sav', 'wb'))

## RandomForest

In [46]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(X_train, Y_train)

y_pred = clf.predict(X_test)
correct = 0
for i in range(len(y_pred)):
    if y_pred[i] == Y_test[i]:
        correct += 1
        
print(correct)
print(len(X_test))
print(correct/len(X_test))

108
206
0.5242718446601942


In [47]:
pickle.dump(clf, open('randomforest_wordnet.sav', 'wb'))